In [ ]:
``

# 산업분류 AI 활용대회

https://data.kostat.go.kr/sbchome/bbs/boardList.do?boardId=SBCSBBS_000000025000&curMenuNo=OPT_09_02_00_0

## BERT의 bert-base-multilingual-uncased 모델

#기본 전처리

##필요한 리소스 준비

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 13.58 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 132 kB 15.3 MB/s 
     |████████████████████████████████| 8.7 MB 90.5 MB/s 
     |████████████████████████████████| 79 kB 10.3 MB/s 
     |████████████████████████████████| 138 kB 99.6 MB/s 
     |████████████████████████████████| 127 kB 96.4 MB/s 
  Created wheel for bert-sklearn: filen

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 13.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595742 sha256=dbfaf76ac6b524426bd0ab59745494524e46fb3717ba14b691e744218f0567a3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 15.8 MB/s 
     |████████████████████████████████| 769 kB 14.5 MB/s 
     |████████████████████████████████| 895 kB 60.8 MB/s 
     |████████████████████████████████| 3.0 MB 59.9 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kj2igzuy
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kj2igzuy
     |████████████████████████████████| 4.5 MB 15.3 MB/s 
     |████████████████████████████████| 4.0 MB 48.5 MB/s 
     |████████████████████████████████| 6.5 MB 62.5 MB/s 
     |████████████████████████████████| 596 kB 76.7 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=cf030dd553f10367f43b91fe6c11215ccf6e7754922a1f9fca28cfda1e65a018
  Stored in directory: /tmp/pip-ephem-wheel-cache-xbz7o95q/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found exist

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import numpy as np
import pandas as pd

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive/')
os.chdir('/content/gdrive/MyDrive/통계청/데이터')

Mounted at /content/gdrive/


##test, train, val file 

###모델1 bert의 인풋

In [ ]:
import pandas as pd
train= pd.read_csv('workdata.txt',sep='|',encoding='euc-kr')
test = pd.read_csv('testdata.txt',sep='|',encoding='euc-kr')

In [ ]:
test['digit_3']=0

In [ ]:
test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,0,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,0,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,0,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,0,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,0,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
test['text_obj'].fillna(" ", inplace = True)
test['text_mthd'].fillna(" ", inplace = True)
test['text_deal'].fillna(" ", inplace = True)
test['data'] = test['text_obj'] + ' ' + test['text_mthd']
test['data'] = test['data'] +  ' ' + test['text_deal']

In [ ]:
train['text_obj'].fillna(" ", inplace = True)
train['text_mthd'].fillna(" ", inplace = True)
train['text_deal'].fillna(" ", inplace = True)
train['data'] = train['text_obj'] + ' ' + train['text_mthd']
train['data'] = train['data'] + ' ' +  train['text_deal']

In [ ]:
train= pd.read_csv('/content/gdrive/MyDrive/통계청/데이터/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/통계청/데이터/test.csv')

###모델2 kobert original data input

In [ ]:
from sklearn.model_selection import train_test_split

train_kb,valid_kb=train_test_split(train, test_size=0.2)

In [ ]:
train_kb.to_csv('/content/gdrive/MyDrive/통계청/데이터/train_kb.tsv', sep='\t',encoding='utf-8',index=False)
valid_kb.to_csv('/content/gdrive/MyDrive/통계청/데이터/valid_kb.tsv', sep='\t',encoding='utf-8',index=False)

In [ ]:
dataset_train = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/train_kb.tsv', field_indices=[7,3], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/valid_kb.tsv', field_indices=[7,3], num_discard_samples=1)

###모델3 kobert 산업분류 추가(2번) data input

In [ ]:
ind_class=pd.read_csv('산업분류표.csv')

In [ ]:
ind_class=ind_class.dropna(how='all')

In [ ]:
ind_class=ind_class.fillna(method='pad')
ind_class

,digit_1,digit_2,digit_3,digit_4,digit_5,type_5
0,A,1.0,11.0,111.0,1110.0,곡물 및 기타 식량작물 재배업
1,A,1.0,11.0,112.0,1121.0,채소작물 재배업
2,A,1.0,11.0,112.0,1122.0,화훼작물 재배업
3,A,1.0,11.0,112.0,1123.0,종자 및 묘목 생산업
4,A,1.0,11.0,113.0,1131.0,과실작물 재배업
...,...,...,...,...,...,...
1192,T,97.0,970.0,9700.0,97000.0,가구 내 고용활동
1193,T,98.0,981.0,9810.0,98100.0,자가 소비를 위한 가사 생산 활동
1194,T,98.0,982.0,9820.0,98200.0,자가 소비를 위한 가사 서비스 활동
1195,U,99.0,990.0,9900.0,99001.0,주한 외국 공관


In [ ]:
train2=train_kb[:0]

In [ ]:
train2['digit_3']=ind_class['digit_3']
train2['data']=ind_class['type_5']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train2['digit_3'] = train2['digit_3'].astype('int64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_kb2=train_kb.append(train2)
train_kb2=train_kb.append(train2)

In [ ]:
train_kb2=train_kb.drop(['AI_id','digit_1','digit_2','text_obj','text_mthd','text_deal'],axis=1)

In [ ]:
valid_kb2=valid_kb.drop(['AI_id','digit_1','digit_2','text_obj','text_mthd','text_deal'],axis=1)

In [ ]:
train_kb2.to_csv('/content/gdrive/MyDrive/통계청/데이터/train_kb5.tsv', sep='\t',encoding='utf-8',index=False)
valid_kb2.to_csv('/content/gdrive/MyDrive/통계청/데이터/valid_kb5.tsv', sep='\t',encoding='utf-8',index=False)

In [ ]:
dataset_train2 = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/train_kb5.tsv', field_indices=[1,0], num_discard_samples=1)
dataset_test2 = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/valid_kb5.tsv', field_indices=[1,0], num_discard_samples=1)

In [ ]:
train.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,data
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서자동차부분정비타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서일반인을 대상으로채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매공업용고무를가지고합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서일반소비자에게열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집보호자의 위탁을 받아취학전아동보육


###모델4 kobert 산업분류 추가(1번) data input

In [ ]:
import pandas as pd
train= pd.read_csv('workdata.txt',sep='|',encoding='euc-kr')
test = pd.read_csv('testdata.txt',sep='|',encoding='euc-kr')

In [ ]:
train['text_obj'].fillna(" ", inplace = True)
train['text_mthd'].fillna(" ", inplace = True)
train['text_deal'].fillna(" ", inplace = True)
train['data'] = train['text_obj'] + ' ' + train['text_mthd']
train['data'] = train['data'] + ' ' +  train['text_deal']

In [ ]:
train=train.append(train2)

In [ ]:
from sklearn.model_selection import train_test_split

train_kb3,valid_kb3=train_test_split(train, test_size=0.2)

In [ ]:
train_kb3.to_csv('/content/gdrive/MyDrive/통계청/데이터/train_kb3.tsv', sep='\t',encoding='utf-8',index=False)
valid_kb3.to_csv('/content/gdrive/MyDrive/통계청/데이터/valid_kb3.tsv', sep='\t',encoding='utf-8',index=False)

In [ ]:
dataset_train3 = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/train_kb3.tsv', field_indices=[7,3], num_discard_samples=1)
dataset_test3 = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/valid_kb3.tsv', field_indices=[7,3], num_discard_samples=1)

#모델 적용

##모델1. bert (with original data)

###모델 학습

In [ ]:
df_train = train[:]

In [ ]:
X = df_train["data"]
y = df_train["digit_3"].values

In [ ]:
model = BertClassifier(bert_model="bert-base-multilingual-uncased", epochs=3, learning_rate=4e-05, validation_fraction=0.2) # 한 에포그 당 40분
model.fit(X, y)

Building sklearn text classifier...
Loading bert-base-multilingual-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 800000, validation data size: 200000



Training  :   0%|          | 0/25000 [00:00<?, ?it/s]

Validating:   0%|          | 0/25000 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.7721, Val loss: 0.3654, Val accy: 90.55%



Training  :   0%|          | 0/25000 [00:00<?, ?it/s]

...


Epoch 2, Train loss: 0.3230, Val loss: 0.3041, Val accy: 91.97%



Training  :   0%|          | 0/25000 [00:00<?, ?it/s]

Validating:   0%|          | 0/25000 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.2427, Val loss: 0.2821, Val accy: 92.50%



BertClassifier(bert_model='bert-base-multilingual-uncased', do_lower_case=True,
               label_list=array([ 11,  12,  14,  20,  31,  32,  51,  61,  62,  71,  72,  80, 101,
       102, 103, 104, 105, 106, 107, 108, 111, 112, 120, 131, 132, 133,
       134, 139, 141, 142, 143, 144, 151, 152, 161, 162, 163, 171, 172,
       179, 181, 182, 191, 192, 201, 202, 203, 204, 205, 211, 212, 213,
       221, 222, 231, 232, 233, 239, 241, 242, 243, 251, 252, 259, 261,
       262, 26...
       521, 529, 551, 559, 561, 562, 581, 582, 591, 592, 601, 602, 611,
       612, 620, 631, 639, 641, 642, 649, 651, 652, 653, 661, 662, 681,
       682, 701, 702, 711, 712, 713, 714, 715, 716, 721, 729, 731, 732,
       733, 739, 741, 742, 743, 751, 752, 753, 759, 761, 762, 763, 764,
       841, 842, 843, 844, 845, 851, 852, 853, 854, 855, 856, 857, 861,
       862, 863, 869, 871, 872, 901, 902, 911, 912, 941, 942, 949, 951,
       952, 953, 961, 969]),
               learning_rate=4e-05, validation_fraction

In [ ]:
import pickle
import joblib

joblib.dump(model, '/content/gdrive/MyDrive/통계청/데이터/bert_2.pkl')  # 모델 저장하기

['/content/gdrive/MyDrive/통계청/데이터/bert_2.pkl']

In [ ]:
model_from_joblib = joblib.load('/content/gdrive/MyDrive/통계청/데이터/bert_2.pkl') # 모델 불러오기
# clf_from_joblib.predict(X)

###학습된 모델 적용

In [ ]:
# 14분 정도 걸림
X_test = test["data"]
y_pred = model.predict(X_test) 

X_test["disit_3"] = y_pred
X_test.head()

Predicting:   0%|          | 0/12500 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0         치킨전문점에서고객의주문에의해치킨판매
1         산업공구다른 소매업자에게철물 수공구
2           절에서신도을 대상으로불교단체운영
3             영업장에서고객요구로자동차튜닝
4    실내포장마차에서접객시설을 갖추고소주,맥주제공
Name: data, dtype: object

In [ ]:
y_pred

array([561, 466, 949, ..., 478, 902, 682])

In [ ]:
X_test = test["data"]
pred_df = pd.DataFrame(X_test)
pred_df["digit_3"] = y_pred
pred_df

,data,digit_3
0,치킨전문점에서고객의주문에의해치킨판매,561
1,산업공구다른 소매업자에게철물 수공구,466
2,절에서신도을 대상으로불교단체운영,949
3,영업장에서고객요구로자동차튜닝,952
4,"실내포장마차에서접객시설을 갖추고소주,맥주제공",562
...,...,...
99995,사업장에서일반인대상으로버섯농장,472
99996,한의원에서외래환자위주고치료,862
99997,일반점포에서소비자에게그림판매,478
99998,사업장에서일반인.학생대상으로학습공간제공,902


In [ ]:
pred_df.to_csv("/content/gdrive/MyDrive/통계청/데이터/bert_2.csv", index = False)

### 소분류를 중,대분류에 매핑

In [ ]:
ind_class=pd.read_csv('산업분류표.csv')

In [ ]:
ind_class=ind_class.drop(['digit_4','digit_5','type_5'],axis=1)

In [ ]:
ind_class=ind_class.dropna(how='all')

In [ ]:
ind_class=ind_class.fillna(method='pad')
ind_class

,digit_1,digit_2,digit_3
0,A,1.0,11.0
10,A,1.0,12.0
17,A,1.0,13.0
18,A,1.0,14.0
21,A,1.0,15.0
...,...,...,...
1181,S,96.0,969.0
1192,T,97.0,970.0
1193,T,98.0,981.0
1194,T,98.0,982.0


In [ ]:
df=pred_df[:]
df_test=df[:]

In [ ]:
df_test

,data,digit_3
0,치킨전문점에서고객의주문에의해치킨판매,561
1,산업공구다른 소매업자에게철물 수공구,466
2,절에서신도을 대상으로불교단체운영,949
3,영업장에서고객요구로자동차튜닝,952
4,"실내포장마차에서접객시설을 갖추고소주,맥주제공",562
...,...,...
99995,사업장에서일반인대상으로버섯농장,472
99996,한의원에서외래환자위주고치료,862
99997,일반점포에서소비자에게그림판매,478
99998,사업장에서일반인.학생대상으로학습공간제공,902


In [ ]:
df['digit_1']=np.nan
df['digit_2']=np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for i in range(len(df)):
  l=ind_class.index[ind_class['digit_3']==df['digit_3'][i]].tolist()
  f=l[0]
  df['digit_1'][i]=ind_class['digit_1'][f]
  df['digit_2'][i]=ind_class['digit_2'][f]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df

,data,digit_3,digit_1,digit_2
0,치킨전문점에서고객의주문에의해치킨판매,561,I,56.0
1,산업공구다른 소매업자에게철물 수공구,466,G,46.0
2,절에서신도을 대상으로불교단체운영,949,S,94.0
3,영업장에서고객요구로자동차튜닝,952,S,95.0
4,"실내포장마차에서접객시설을 갖추고소주,맥주제공",562,I,56.0
...,...,...,...,...
99995,사업장에서일반인대상으로버섯농장,472,G,47.0
99996,한의원에서외래환자위주고치료,862,Q,86.0
99997,일반점포에서소비자에게그림판매,478,G,47.0
99998,사업장에서일반인.학생대상으로학습공간제공,902,R,90.0


In [ ]:
df.to_csv("/content/gdrive/MyDrive/통계청/데이터/제출1_bertfull3.csv", index = False)

In [ ]:
ans = pd.read_csv('답안 작성용 파일.csv',encoding='euc-kr')

In [ ]:
ans['digit_1']=df['digit_1']
ans['digit_2']=df['digit_2']
ans['digit_3']=df['digit_3']

In [ ]:
ans

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95.0,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,47.0,472,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
ans.to_csv("/content/gdrive/MyDrive/통계청/데이터/제출1_bertfull3.csv", index = False)

## 모델2. KOBERT (with original data) 

###모델 학습

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_v1.zip
using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#파라미터 튜닝..?
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=990, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/12500 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.939396381378174 train acc 0.0
epoch 1 batch id 201 loss 6.7622761726379395 train acc 0.0017101990049751244
epoch 1 batch id 401 loss 6.095229625701904 train acc 0.050810473815461346
epoch 1 batch id 601 loss 5.320389270782471 train acc 0.09299604825291181
epoch 1 batch id 801 loss 3.662550926208496 train acc 0.11657303370786516
epoch 1 batch id 1001 loss 3.5258686542510986 train acc 0.14327859640359641
epoch 1 batch id 1201 loss 2.8083581924438477 train acc 0.17909554537885095
epoch 1 batch id 1401 loss 2.6241750717163086 train acc 0.21872769450392576
epoch 1 batch id 1601 loss 2.502591371536255 train acc 0.25892996564647097
epoch 1 batch id 1801 loss 2.0336897373199463 train acc 0.2978553581343698
epoch 1 batch id 2001 loss 1.9714664220809937 train acc 0.3344421539230385
epoch 1 batch id 2201 loss 1.9008574485778809 train acc 0.3668573943661972
epoch 1 batch id 2401 loss 1.684599757194519 train acc 0.3970220741357768
epoch 1 batch id 2601 loss 1.5829514265060

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3125 [00:00<?, ?it/s]

...

epoch 4 validation acc 0.92861


  0%|          | 0/12500 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.18010176718235016 train acc 0.96875
epoch 5 batch id 201 loss 0.19482605159282684 train acc 0.9489272388059702
epoch 5 batch id 401 loss 0.16796816885471344 train acc 0.9481374688279302
epoch 5 batch id 601 loss 0.1444590836763382 train acc 0.9497972129783694
epoch 5 batch id 801 loss 0.1673233062028885 train acc 0.9493406679151061
epoch 5 batch id 1001 loss 0.11450120806694031 train acc 0.9498626373626373
epoch 5 batch id 1201 loss 0.014697805978357792 train acc 0.9500026019983348
epoch 5 batch id 1401 loss 0.2495439499616623 train acc 0.9501695217701641
epoch 5 batch id 1601 loss 0.10293184965848923 train acc 0.9503728138663335
epoch 5 batch id 1801 loss 0.10365220904350281 train acc 0.9503661160466408
epoch 5 batch id 2001 loss 0.08854091167449951 train acc 0.9507589955022488
epoch 5 batch id 2201 loss 0.506439208984375 train acc 0.9506190368014539
epoch 5 batch id 2401 loss 0.2988743185997009 train acc 0.9504308100791337
epoch 5 batch id 2601 loss 0.164909

  0%|          | 0/3125 [00:00<?, ?it/s]

epoch 5 validation acc 0.928945


###학습된 모델 적용

In [ ]:
new_test = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/test.tsv', field_indices=[7,3], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
prds=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  prds.append(prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
ans = pd.read_csv('답안 작성용 파일.csv',encoding='euc-kr')

In [ ]:
for i in range(len(ans)):
  ans['digit_3'][i]=prds[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###소분류를 중,대분류에 매핑

In [ ]:
df=ans[:]

In [ ]:
for i in range(len(df)):
  l=ind_class.index[ind_class['digit_3']==df['digit_3'][i]].tolist()
  f=l[0]
  df['digit_1'][i]=ind_class['digit_1'][f]
  df['digit_2'][i]=ind_class['digit_2'][f]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/li

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95.0,952.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,A,1.0,11.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862.0,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478.0,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902.0,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
df.to_csv("/content/gdrive/MyDrive/통계청/데이터/제출2_kobertfull5.csv", index = False)

## 모델3. KOBERT (with added data twice) 

###모델 학습

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_v1.zip
using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#파라미터 튜닝..?
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train2, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test2, 0, 1, tok, max_len, True, False)

using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=991, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/12538 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.957755088806152 train acc 0.0
epoch 1 batch id 201 loss 6.73248815536499 train acc 0.0021766169154228856
epoch 1 batch id 401 loss 6.3135528564453125 train acc 0.04547225685785536
epoch 1 batch id 601 loss 5.711589813232422 train acc 0.07903494176372712
epoch 1 batch id 801 loss 4.857412338256836 train acc 0.10520053058676654
epoch 1 batch id 1001 loss 4.296240329742432 train acc 0.1263267982017982
epoch 1 batch id 1201 loss 3.662010431289673 train acc 0.1528023522064946
epoch 1 batch id 1401 loss 3.1565725803375244 train acc 0.18315042826552463
epoch 1 batch id 1601 loss 2.9369664192199707 train acc 0.21608564959400375
epoch 1 batch id 1801 loss 2.38822865486145 train acc 0.25142282065519156
epoch 1 batch id 2001 loss 2.394650936126709 train acc 0.28612256371814093
epoch 1 batch id 2201 loss 1.8532437086105347 train acc 0.31751192639709225
epoch 1 batch id 2401 loss 1.3464562892913818 train acc 0.34587151187005416
epoch 1 batch id 2601 loss 1.6012732982635498

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3125 [00:00<?, ?it/s]

...

epoch 6 validation acc 0.92756


  0%|          | 0/12538 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.2231331318616867 train acc 0.953125
epoch 7 batch id 201 loss 0.10579100996255875 train acc 0.9578669154228856
epoch 7 batch id 401 loss 0.09960614144802094 train acc 0.9600997506234414
epoch 7 batch id 601 loss 0.13018931448459625 train acc 0.9595465890183028
epoch 7 batch id 801 loss 0.1456485241651535 train acc 0.9589380461922596
epoch 7 batch id 1001 loss 0.24266768991947174 train acc 0.9594468031968032
epoch 7 batch id 1201 loss 0.0705498680472374 train acc 0.959421835970025
epoch 7 batch id 1401 loss 0.338710755109787 train acc 0.9597051213418987
epoch 7 batch id 1601 loss 0.2996182441711426 train acc 0.9595174890693317
epoch 7 batch id 1801 loss 0.19872719049453735 train acc 0.9596318017767906
epoch 7 batch id 2001 loss 0.11951407790184021 train acc 0.9599809470264867
epoch 7 batch id 2201 loss 0.15994364023208618 train acc 0.9600039754656974
epoch 7 batch id 2401 loss 0.05803202465176582 train acc 0.9600296751353603
epoch 7 batch id 2601 loss 0.2391698

  0%|          | 0/3125 [00:00<?, ?it/s]

epoch 7 validation acc 0.928145


###학습된 모델 적용

In [ ]:
new_test = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/test.tsv', field_indices=[7,3], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
prds=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  prds.append(prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
ans = pd.read_csv('답안 작성용 파일.csv',encoding='euc-kr')

In [ ]:
for i in range(len(ans)):
  ans['digit_3'][i]=prds[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###소분류를 중,대분류에 매핑

In [ ]:
df=ans[:]

In [ ]:
for i in range(len(df)):
  l=ind_class.index[ind_class['digit_3']==df['digit_3'][i]].tolist()
  f=l[0]
  df['digit_1'][i]=ind_class['digit_1'][f]
  df['digit_2'][i]=ind_class['digit_2'][f]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/li

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,C,30.0,302.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,46.0,463.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862.0,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478.0,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902.0,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
df.to_csv("/content/gdrive/MyDrive/통계청/데이터/제출3_kobertfull7.csv", index = False)

## 모델4. KOBERT (with added data once) 

###모델 학습

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_v1.zip
using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#파라미터 튜닝..?
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train3, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test3, 0, 1, tok, max_len, True, False)

using cached model. /content/gdrive/MyDrive/통계청/데이터/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=991, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25038 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.916141986846924 train acc 0.0
epoch 1 batch id 201 loss 6.915793418884277 train acc 0.0001554726368159204
epoch 1 batch id 401 loss 6.823721885681152 train acc 0.0020261845386533666
epoch 1 batch id 601 loss 6.501308917999268 train acc 0.01752287853577371
epoch 1 batch id 801 loss 6.316479682922363 train acc 0.045177902621722846
epoch 1 batch id 1001 loss 5.993305206298828 train acc 0.06993006993006994
epoch 1 batch id 1201 loss 5.776043891906738 train acc 0.08976894254787678
epoch 1 batch id 1401 loss 4.674968242645264 train acc 0.1065087437544611
epoch 1 batch id 1601 loss 5.267349720001221 train acc 0.11972985633978764
epoch 1 batch id 1801 loss 4.145011901855469 train acc 0.12996252082176568
epoch 1 batch id 2001 loss 4.1780924797058105 train acc 0.13974262868565718
epoch 1 batch id 2201 loss 4.486166477203369 train acc 0.14988925488414356
epoch 1 batch id 2401 loss 3.9356887340545654 train acc 0.1620158267388588
epoch 1 batch id 2601 loss 3.95995974540710

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6250 [00:00<?, ?it/s]

...

epoch 9 validation acc 0.926295


  0%|          | 0/25038 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.26175621151924133 train acc 0.9375
epoch 10 batch id 201 loss 0.537787675857544 train acc 0.9752798507462687
epoch 10 batch id 401 loss 0.2115953117609024 train acc 0.9743609725685786
epoch 10 batch id 601 loss 0.005792762152850628 train acc 0.9727017470881864
epoch 10 batch id 801 loss 0.3165817856788635 train acc 0.9730024968789014
epoch 10 batch id 1001 loss 0.1187845915555954 train acc 0.9723401598401599
epoch 10 batch id 1201 loss 0.05797606706619263 train acc 0.9724708576186512
epoch 10 batch id 1401 loss 0.18614976108074188 train acc 0.9728988222698073
epoch 10 batch id 1601 loss 0.20468424260616302 train acc 0.9724000624609619
epoch 10 batch id 1801 loss 0.02793189510703087 train acc 0.9722029428095502
epoch 10 batch id 2001 loss 0.1120668351650238 train acc 0.9720608445777111
epoch 10 batch id 2201 loss 0.3355885446071625 train acc 0.9720297592003635
epoch 10 batch id 2401 loss 0.20434026420116425 train acc 0.9721990837151187
epoch 10 batch id 2601 l

  0%|          | 0/6250 [00:00<?, ?it/s]

epoch 10 validation acc 0.92626


###학습된 모델 적용

In [ ]:
new_test = nlp.data.TSVDataset('/content/gdrive/MyDrive/통계청/데이터/test.tsv', field_indices=[7,3], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
prds=[]
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  prds.append(prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
ans = pd.read_csv('답안 작성용 파일.csv',encoding='euc-kr')

In [ ]:
for i in range(len(ans)):
  ans['digit_3'][i]=prds[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###소분류를 중,대분류에 매핑

In [ ]:
df=ans[:]

In [ ]:
for i in range(len(df)):
  l=ind_class.index[ind_class['digit_3']==df['digit_3'][i]].tolist()
  f=l[0]
  df['digit_1'][i]=ind_class['digit_1'][f]
  df['digit_2'][i]=ind_class['digit_2'][f]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/li

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95.0,952.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,46.0,463.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862.0,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478.0,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902.0,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
df.to_csv("/content/gdrive/MyDrive/통계청/데이터/제출5_kobertfull10.csv", index = False)

#앙상블

In [ ]:
import pandas as pd 

In [ ]:
ans1=pd.read_csv('제출2_kobertfull5.csv') #가장 높은 val acc
ans2=pd.read_csv('제출3_kobertfull7.csv')
ans3=pd.read_csv('제출1_bertfull3.csv')
ans4=pd.read_csv('제출5_kobertfull10.csv')

In [ ]:
m1=[]
for i in ans1['digit_3']:
  m1.append(i)

In [ ]:
m2=[]
for i in ans2['digit_3']:
  m2.append(i)

In [ ]:
m3=[]
for i in ans3['digit_3']:
  m3.append(i)

In [ ]:
m4=[]
for i in ans4['digit_3']:
  m4.append(i)

In [ ]:
ensemble = [] 

for a, b, c, d in zip(m1, m2, m3, m4): 
    if b==c==d :
      ensemble.append(b)
    else:
      ensemble.append(a)


In [ ]:
for i in range(len(ans1)):
  ans1['digit_3'][i]=ensemble[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
ind_class=pd.read_csv('산업분류표.csv')

In [ ]:
ind_class=ind_class.drop(['digit_4','digit_5','type_5'],axis=1)

In [ ]:
ind_class=ind_class.dropna(how='all')

In [ ]:
ind_class=ind_class.fillna(method='pad')
ind_class

,digit_1,digit_2,digit_3
0,A,1.0,11.0
10,A,1.0,12.0
17,A,1.0,13.0
18,A,1.0,14.0
21,A,1.0,15.0
...,...,...,...
1181,S,96.0,969.0
1192,T,97.0,970.0
1193,T,98.0,981.0
1194,T,98.0,982.0


In [ ]:
df=ans1[:]

In [ ]:
for i in range(len(df)):
  l=ind_class.index[ind_class['digit_3']==df['digit_3'][i]].tolist()
  f=l[0]
  df['digit_1'][i]=ind_class['digit_1'][f]
  df['digit_2'][i]=ind_class['digit_2'][f]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
ans = pd.read_csv('답안 작성용 파일.csv',encoding='euc-kr')

In [ ]:
ans['digit_1']=df['digit_1']
ans['digit_2']=df['digit_2']
ans['digit_3']=df['digit_3']

In [ ]:
ans

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95.0,952.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,A,1.0,11.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862.0,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478.0,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902.0,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
ans.to_csv("/content/gdrive/MyDrive/통계청/데이터/제출fianl.csv", index = False)